In [1]:
import json
import random

from agumentations import *
from tqdm.notebook import tqdm

import torch

from transformers import MarianMTModel, MarianTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

/home/phucphan/anaconda3/envs/phucpx/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


No GPU available, using the CPU instead.
No GPU available, using the CPU instead.


In [2]:
name_file = 'train-9'
with open(f'./uit-visquad/{name_file}.json', 'r+', encoding='utf-8') as f:
    data = json.load(f)

In [3]:
data_ = data['data']
len(data_)

125

In [4]:
tar_model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-vi-en-base")
tar_tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-vi-en-base")
tar_model.to(device)

enc_model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-en-vi-base")
enc_tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-en-vi-base")
enc_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (w

In [6]:
for i in tqdm(range(len(data_))):
    paragraphs = data_[i]['paragraphs']
    for par in paragraphs:
        qas = par['qas']
        len_qas = len(qas)
        for j in range(len_qas):
            q = qas[j]
            tmp = {
                'id': q['id'] + str(j), 
                'answers': q['answers'],
                'is_impossible': q['is_impossible']
            }
            question = q['question']
            
            aug_text = translate(question, tar_model, tar_tokenizer, enc_model, enc_tokenizer)
            tmp['question'] = aug_text
            
            if aug_text.lower() != question.lower():
                qas.append(aug_text)

KeyboardInterrupt: 

In [ ]:
with open(f'./uit-visquad/train-aug-9.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)